# Running FMRIprep on the GRMPY Dataset

In this notebook we document running `fmriprep` on the full grmpy data set. 

First, query flywheel for the gear

In [1]:
import flywheel

In [2]:
fw = flywheel.Client()

In [3]:
fmriprep = fw.lookup('gears/fmriprep125')

Interestingly, the UPENN instance has its `fmriprep` listed as `fmriprep125`. Here is the gear object:

In [4]:
fmriprep

{'category': 'analysis',
 'created': datetime.datetime(2019, 2, 13, 19, 26, 14, 105000, tzinfo=tzutc()),
 'exchange': {'git_commit': None,
              'rootfs_hash': None,
              'rootfs_url': 'docker://upenn.flywheel.io/fmriprep125@sha256:a1ba2dab2c2758fde56f2d58a0974702693e1a03998ba72d0486f3973d283758'},
 'gear': {'author': 'Poldrack Lab, Stanford University',
          'cite': 'Esteban O, Blair R, Markiewicz CJ, Berleant SL, Moodie C, '
                  'Ma F, Isik AI, Erramuzpe A, Kent JD, Goncalves M, Poldrack '
                  'RA, Gorgolewski KJ; poldracklab/fmriprep: 1.0.0-rc9. '
                  'Zenodo; 2017. doi:10.5281/zenodo.1041020  FOR MORE '
                  'INFORMATION SEE: '
                  'http://fmriprep.readthedocs.io/en/stable/citing.html.',
          'command': None,
          'config': {'FREESURFER_LICENSE': {'description': 'Text from license '
                                                           'file generated '
                        

You can see that there are some non-optional arguments in the `inputs`. We will have to set these. Otherwise, it is just a matter of running the gear as demonstrated in the [docs](https://flywheel-io.github.io/core/branches/master/python/examples.html#jobs-and-analyses). 

First, we write a function to find the T1w scan in a session and return it.

In [5]:
proj = fw.projects.find_first("label=GRMPY_822831")
# an example session
ses = proj.sessions()[25]

In [6]:
def pull_t1w(session):
    
    acquisitions = session.acquisitions()
    files = [y for x in acquisitions for y in x.files]
    for f in files:
        if isinstance(f.info['BIDS'], dict):
            if 'Modality' in list(f.info['BIDS'].keys()):
                if f.info['BIDS']['Modality'] == "T1w":
                    return f

In [7]:
pull_t1w(ses).name

'MPRAGE_TI1100_ipat2_2.nii.gz'

Then, we set the config. We have a couple of booleans and the `ignore` slicetiming option that we have to set, but we have to input the text of the freesurfer license at this stage:

In [8]:
with open("license.txt") as f: 
    freesurfer_lic = f.read() 

In [9]:
freesurfer_lic

'stauffer@upenn.edu\n20217\n *CNJKge/MKN62\n FShMG3atCAhWE\n'

In [10]:
config = {
    
    'FREESURFER_LICENSE': freesurfer_lic,
    "cifti_output": True,
    "force_bbr": True,
    "ignore": "slicetiming"

}

For the input, we can use the project's license file.

In [17]:
inputs = {
    
    "freesurfer_license": proj.files[-1],
    "t1w_anatomy": pull_t1w(ses)
}

And then we run:

In [18]:
analysis_id = fmriprep.run(analysis_label='fmriprep_SDK_demo', config=config, inputs=inputs, destination=ses)

In [19]:
analysis_id

'5cb4e961f546b60027ef6746'

In [20]:
ses

{'age': 788400000,
 'analyses': [],
 'created': datetime.datetime(2019, 2, 25, 19, 24, 2, 521000, tzinfo=tzutc()),
 'files': [{'classification': {},
            'created': datetime.datetime(2019, 2, 25, 19, 26, 2, 172000, tzinfo=tzutc()),
            'hash': '',
            'id': '7dd9920c-67c7-466e-afa8-aae8577bb1be',
            'info': {'BIDS': {'Filename': '',
                              'Folder': 'ses-010069',
                              'Path': 'sub-010069/ses-010069',
                              'error_message': "Filename '' is too short",
                              'ignore': False,
                              'template': 'session_file',
                              'valid': False}},
            'info_exists': None,
            'mimetype': 'text/xml',
            'modality': None,
            'modified': datetime.datetime(2019, 3, 27, 19, 51, 14, 28000, tzinfo=tzutc()),
            'name': '010069_20160217_163345.xml',
            'origin': {'id': 'harshakethineni@fl

Now, we loop:

In [22]:
fails = []
for ses in proj.sessions():

    try:
        config = {
    
            'FREESURFER_LICENSE': freesurfer_lic,
            "cifti_output": True,
            "force_bbr": True,
            "ignore": "slicetiming"

        }
        
        inputs = {
    
            "freesurfer_license": proj.files[-1],
            "t1w_anatomy": pull_t1w(ses)
            
        }
        analysis_id = fmriprep.run(analysis_label='fmriprep_SDK_{}'.format(ses.label), config=config, inputs=inputs, destination=ses)
    except Exception as e:
        print(e)
        fails.append(ses)

'NoneType' object has no attribute 'ref'
'NoneType' object has no attribute 'ref'
'NoneType' object has no attribute 'ref'
'NoneType' object has no attribute 'ref'
'NoneType' object has no attribute 'ref'
'NoneType' object has no attribute 'ref'
'NoneType' object has no attribute 'ref'
'BIDS'


In [24]:
len(fails)

8

In [25]:
fails[0]

{'age': None,
 'analyses': None,
 'created': datetime.datetime(2019, 2, 26, 16, 2, 23, 270000, tzinfo=tzutc()),
 'files': [{'classification': {},
            'created': datetime.datetime(2019, 2, 26, 16, 4, 58, 32000, tzinfo=tzutc()),
            'hash': '',
            'id': '9a03ab9a-18e8-404a-b719-ac7ac9ca96ee',
            'info': {'BIDS': {'Filename': '',
                              'Folder': 'ses-010333',
                              'Path': 'sub-010333/ses-010333',
                              'error_message': "Filename '' is too short",
                              'ignore': False,
                              'template': 'session_file',
                              'valid': False}},
            'info_exists': True,
            'mimetype': 'application/octet-stream',
            'modality': None,
            'modified': datetime.datetime(2019, 3, 27, 19, 53, 35, 86000, tzinfo=tzutc()),
            'name': 'dcmtoxnat.log',
            'origin': {'id': 'harshakethineni@fly